# Business Case Study - Staff Planning

# Problem Statement

 The insurance approval process, i.e., underwriting is one of the important and time-consuming tasks in an insurance application processing. When you submit your insurance application, the underwriter of the company evaluates it based on the details that you provide using a rule-based or an ML model and decides whether or not to approve your application. An insurance company InsurePlus wants you to help them with finding the optimal number of staff that they need for their insurance application approval process for the calendar year 2021. In the industry, the number of staffs is considered as a continuous variable. This is also called a staff Full-Time Equivalent. For example, if a full-time employee (FTE =1) works for 50 hours a week, 10 hours corresponds to 0.2 FTEs.

- The company operates in three states A, B, and C.
- The company can either handle an application with the staff that they hire or outsource it to a vendor. (Assume that there is no capacity limitation to outsourcing.)
- If they hire staff, he/she can handle 40 insurance applications per month when he/she works 100% of the workdays. However, there are days that he/she will be unavailable to process applications due to training, off days, etc.
- A staff member’s availability (in percentage) to work on processing the insurance applications for each month is shown in the table given below. As mentioned before, with 100% availability, each member can handle 40 applications.

A special note of practical relevance: In the industry, staff availability is predicted using a time-motion study. But in this case, you have been given fixed numbers for each month. States A and B have a regulatory restriction that the outsourced insurance applications cannot be more than 30% and 40% of the total number of applications for each month, respectively. The objective is to optimise the total cost for the application approval process by distributing the right number of applications between the FTEs and the vendors while meeting the monthly demand for each state at the same time.



## Importing the libraries and reading the dataset

In [59]:
# Importing Libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from pyomo.environ import *

In [60]:
from __future__ import division
from pyomo.opt import SolverFactory

In [61]:
# Reading the data from Excel workbook - Staffing.xlsx

InputData = 'Staffing+Data.xlsx'

#Read the data from Demand Data sheet

Demand = pd.read_excel(InputData,sheet_name='DemandData')

#Read the data from Staff Availability sheet

Availability = pd.read_excel (InputData,sheet_name='StaffAvailability')

#Read the data from Cost sheet

Cost = pd.read_excel (InputData,sheet_name='Cost')

#Max Applications Handled by FTE per month

ServiceRate = pd.read_excel( InputData, sheet_name ='ServiceRate')

In [62]:
#Printing the dataframe Demand

Demand.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [63]:
#Printing the dataframe Availability

Availability.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [64]:
#Printing the dataframe Cost

Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [65]:
#Printing the dataframe ServiceRate

ServiceRate

,MgAppServedPerMonth
0,40


In [66]:
# Consolidating the three dataframes into single dataframe for further analysis

data=pd.merge(Demand,Availability ,on=['State','Month'])

Staff=pd.merge(data,Cost, on = ['State','Month'])

Staff.head()

,State,Month,Demand,LB,UB,StaffAvPer,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,5240,0.70,0.90,0.81,60000,5000.0,180
1,A,Feb,4878,0.65,0.85,0.76,60000,5000.0,180
2,A,Mar,5942,0.70,0.80,0.75,60000,5000.0,180
3,A,Apr,2297,0.75,0.85,0.80,60000,5000.0,180
4,A,May,1992,0.70,0.85,0.78,60000,5000.0,180


In [67]:
#Checking the Shape of Final dataframe

Staff.shape

(36, 9)

In [68]:
#Checking if any null values present in the final dataframe

Staff.isnull().sum()

State                0
Month                0
Demand               0
LB                   0
UB                   0
StaffAvPer           0
AnnualSalary         0
MonthlySalary        0
UnitOutSourceCost    0
dtype: int64

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



##  Mathematical formulation / Pyomo components

---

**Sets:** 

The indexes for the given problem are, <br>
- State, $i \in State$
- Month, $j \in Month$

---

**Parameters:** 

- $\text{x}_{i,j} \text{ - Applications processed by FTE}$ <br>


- $\text{y}_{i,j} \text{ - Applications outsourced}$ <br>


- $\text{FTE_Salary}_{i,j} \text{ - Monthly Salary of FTE}$ <br>


- $\text{UnitOutSourceCost}_{i,j} \text{ - Unit Cost for outsourcing 1 application}$ <br>


- $\text{StaffAvPer}_{i,j} \text{ - Average availability of FTE}$ <br>


- $\text{StaffAv_LB}_{i,j} \text{ - Minimum availability of FTE}$ <br>


- $\text{StaffAv_UB}_{i,j} \text{ - Maximum availability of FTE}$ <br>


- $\text{FTE_AppServRate} \text{ - Number of insurance applications that can be processed by an FTE in a month when working with 100 percent availability}$ <br>
                                                                                                                                                                                                                   
---

**Decision variables:**
                                                                                                             
- Total number of  applications processed by FTE for each state and month combination <br>

 $\text{x}_{i,j} \,\,\,\,\,\,\, \text{where} \ i \in State, j \in Month $<br>
 
 
- Total number of  applications outsourced for each state and month combination <br>

 $\text{y}_{i,j} \,\,\,\,\,\,\, \text{where} \ i \in State, y \in Month $<br>


---

**Objective Function:** <br>

To minimize the total cost of insurance approval process by distributing the right number of applications between FTE and outsourced.



\begin{align}
\textrm{min} (\sum \limits _{i,j} \text{x}_{i,j}* \text{FTE_Salary}_{i,j} + \sum \limits _{i,j} \text{y}_{i,j}* \text{UnitOutSourceCost}_{i,j})
\end{align}
where $i \in State$ and $j \in Month$

---

**Constraints:**


- **Demand Constraint** :- Demand of Insurance for each state and month combination


>- $\sum \limits _{i,j} \text{x}_{i,j}* \text{StaffAvPer}_{i,j}*\text{FTE_AppServRate} == \text{demand} \ \ \ \ \forall  {i \in State, j \in Month}$


- **Staff Availability** :- Staff Availability for best and worst case scenerios.


>- $ \text{StaffAv_LB}_{i,j}  > \text{lb}_{i,j} \ \ \ \ \forall  {i \in State, j \in Month}$

>- $ \text{StaffAv_UB}_{i,j}  < \text{ub}_{i,j} \ \ \ \ \forall  {i \in State, j \in Month}$



- **Outsourcing Regulations** :- Regulatory Regulations on States A & B for outsourcing applications.

>- $ \text{y}_{i,j}  < (\text0.3*\text{demand}_{i,j}) \,\,\,\,\,\,\, \text{where} \ i \in A , j \in Month $

>- $ \text{y}_{i,j}  < (\text0.4*\text{demand}_{i,j}) \,\,\,\,\,\,\, \text{where} \ i \in B , j \in Month $


# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

## Data pre-processing 

In [69]:
# Create the required Python data structures for indexes and parameters

#Extracting States into a list

State = Staff['State'].unique()

# Extracting Months into a list

Month = Staff['Month'].unique()

# No of applications processed when FTE is 100% available

MaxAppFTE = ServiceRate['MgAppServedPerMonth']

# Create a Dictionary object Demand with index: 'State' and 'Month' and Value as "Demand" column from the Staff Dataframe

Demand = Staff.set_index(['State','Month'])['Demand'].to_dict()

# Create a Dictionary object FTE_Monthly_Salary with index: 'State' and 'Month' and Value as "MonthlySalary" column 
#from the Staff Dataframe

FTE_Monthly_Salary = Staff.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Create a Dictionary object UnitOutSourceCost with index: 'State' and 'Month' and Value as "UnitOutSourceCost" column 
#from the Staff Dataframe

UnitOutSourceCost = Staff.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Create a Dictionary object Staff_Availability with index: 'State' and 'Month' and Value as "StaffAvPer" column 
#from the Staff Dataframe

Staff_Availability = Staff.set_index(['State','Month'])['StaffAvPer'].to_dict()

# Create a Dictionary object Staff_Availability_LB with index: 'State' and 'Month' and Value as "LB" column 
#from the Staff Dataframe

Staff_Availability_LB = Staff.set_index(['State','Month'])['LB'].to_dict()

# Create a Dictionary object Staff_Availability_UB with index: 'State' and 'Month' and Value as "UB" column 
#from the Staff Dataframe

Staff_Availability_UB = Staff.set_index(['State','Month'])['UB'].to_dict()

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [70]:
# Creating a model instance

model = ConcreteModel()

In [71]:
# Define Pyomo sets and Parameters

model.s = Set(initialize=State.tolist(),doc='States')

model.m = Set(initialize=Month.tolist(),doc='Months')

model.demand = Param(model.s,model.m,initialize = Demand,doc='Demand')

model.sa = Param(model.s,model.m,initialize = Staff_Availability,doc = 'Staff_Availability')

In [72]:
# Decision variables

model.FTE = Var(model.s,model.m,doc='No of FTE',domain = NonNegativeReals)

model.Outsource = Var(model.s,model.m,doc='No of Outsource App',domain = NonNegativeIntegers)

In [73]:
# Constraints

# Demand Constraint

def total_demand(M,s,m):
    return (M.FTE[s,m]*40*M.sa[s,m] + M.Outsource[s,m] == M.demand[s,m])
            
model.total_demand = Constraint(model.s,model.m, rule=total_demand)

# Outsourcing Regulation

model.outsource_demand = ConstraintList()
for m in model.m:
    for s in model.s:
        if s == 'A':
            model.outsource_demand.add(expr = model.Outsource[s,m] <= 0.3*model.demand[s,m])
        elif s == 'B':
            model.outsource_demand.add(expr = model.Outsource[s,m] <= 0.4*model.demand[s,m])

In [74]:
# Objective function

model.value = Objective(expr = sum(sum(model.FTE[s,m]*FTE_Monthly_Salary[s,m]
                                       + model.Outsource[s,m]*UnitOutSourceCost[s,m] for s in model.s) for m in model.m), 
                        sense= minimize)

In [75]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07255887985229492
# ----------------------------------------------------------
#   Solution Information
# ------------

In [76]:
#Printing the output of the Model

model.pprint()

8 Set Declarations
    FTE_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    s*m :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Outsource_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    s*m :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B

In [77]:
# Print the value of the objective function

optimized_cost = round(model.value()/1000000,2)
optimized_cost

17.96

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [78]:
#Creating an empty list

Output_Optimised = []

# For every iteration, the code below will create a list containing 9 values - State, Month, Demand,No of FTE,
#No of Outsource,No of FTE Processed Applications,Percent FTE Applications,Percent Outsourced Applications,Estimated Cost,
#Average Cost per Application

for s in model.s:
    for m in model.m:
        
        
        Demand = model.demand[s, m]
        No_of_FTE=model.FTE[s,m].value
        No_of_Outsource = model.Outsource[s,m].value
        No_of_FTE_Processed_App = model.FTE[s,m].value*40*model.sa[s,m]
        Percent_FTE_App = round((No_of_FTE/Demand)*100,2)
        Percent_Outsourced_App = round((No_of_Outsource/Demand)*100,2)
        Estimated_Cost = model.FTE[s,m].value*FTE_Monthly_Salary[s,m] + model.Outsource[s,m].value*UnitOutSourceCost[s,m]
        Cost_Per_App = round(Estimated_Cost/Demand,2)
        
        Output_Optimised.append([s,m,Demand,No_of_FTE,No_of_Outsource,No_of_FTE_Processed_App,Percent_FTE_App, 
                              Percent_Outsourced_App,Estimated_Cost,Cost_Per_App])
        
print(Output_Optimised)

[['A', 'Jan', 5240, 161.728395061728, 0.0, 5239.999999999987, 3.09, 0.0, 808641.97530864, 154.32], ['A', 'Feb', 4878, 160.460526315789, 0.0, 4877.999999999985, 3.29, 0.0, 802302.631578945, 164.47], ['A', 'Mar', 5942, 198.066666666667, 0.0, 5942.000000000011, 3.33, 0.0, 990333.333333335, 166.67], ['A', 'Apr', 2297, 71.78125, 0.0, 2297.0, 3.12, 0.0, 358906.25, 156.25], ['A', 'May', 1992, 63.8461538461538, 0.0, 1991.9999999999986, 3.21, 0.0, 319230.76923076896, 160.26], ['A', 'Jun', 2275, 77.9109589041096, 0.0, 2275.0000000000005, 3.42, 0.0, 389554.794520548, 171.23], ['A', 'Jul', 5334, 137.279411764706, 1600.0, 3734.000000000003, 2.57, 30.0, 974397.05882353, 182.68], ['A', 'Aug', 3371, 110.888157894737, 0.0, 3371.0000000000055, 3.29, 0.0, 554440.7894736851, 164.47], ['A', 'Sep', 3759, 116.018518518519, 0.0, 3759.000000000016, 3.09, 0.0, 580092.592592595, 154.32], ['A', 'Oct', 3529, 120.856164383562, 0.0, 3529.0000000000105, 3.42, 0.0, 604280.82191781, 171.23], ['A', 'Nov', 4284, 110.2573

In [79]:
# Creating dataframe for the results

Output_Optimised = pd.DataFrame(Output_Optimised, columns = ['State','Month','Demand','No of FTE','No of Outsource',
                                                       'No of FTE Processed Applications','Percent FTE Applications',
                                                       'Percent Outsourced Applications','Estimated Cost',
                                                       'Cost per Application'])

Output_Optimised[['Estimated Cost in mn $']] = Output_Optimised[['Estimated Cost']]/1000000

Output_Optimised['Demand'] = Output_Optimised['Demand'].astype(int)

Output_Optimised['No of Outsource'] = Output_Optimised['No of Outsource'].astype(int)

Output_Optimised['No of FTE Processed Applications'] = Output_Optimised['No of FTE Processed Applications'].astype(int)

Output_Optimised = Output_Optimised.round({"No of FTE":1, "No of Outsource":0, "Estimated Cost in mn $":1})

In [80]:
#Printing top 5 Rows of Final dataframe

Output_Optimised.head()

,State,Month,Demand,No of FTE,No of Outsource,No of FTE Processed Applications,Percent FTE Applications,Percent Outsourced Applications,Estimated Cost,Average Cost per Application,Estimated Cost in mn $
0,A,Jan,5240,161.7,0,5239,3.09,0.0,808641.975309,154.32,0.8
1,A,Feb,4878,160.5,0,4877,3.29,0.0,802302.631579,164.47,0.8
2,A,Mar,5942,198.1,0,5942,3.33,0.0,990333.333333,166.67,1.0
3,A,Apr,2297,71.8,0,2297,3.12,0.0,358906.250000,156.25,0.4
4,A,May,1992,63.8,0,1991,3.21,0.0,319230.769231,160.26,0.3


In [81]:
# Writing the results in to an Excel sheet

with pd.ExcelWriter('Output_Optimised_Q2.xlsx') as writer:
    Output_Optimised.to_excel(writer)

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [82]:
# Creating a model instance

model1 = ConcreteModel()

In [83]:
# Define Pyomo sets and Parameters


model1.s = Set(initialize=State.tolist(),doc='States')

model1.m = Set(initialize=Month.tolist(),doc='Months')

model1.demand = Param(model1.s,model1.m,initialize = Demand,doc='Demand')

model1.sa = Param(model1.s,model1.m,initialize = Staff_Availability_LB,doc = 'Staff_Availability_LB')

In [84]:
# Decision variables

model1.FTE = Var(model1.s,model1.m,doc='No of FTE',domain = NonNegativeReals)

model1.Outsource = Var(model1.s,model1.m,doc='No of Outsource App',domain = NonNegativeIntegers)

In [85]:
# Constraints

def total_demand(M,s,m):
    return (M.FTE[s,m]*40*M.sa[s,m] + M.Outsource[s,m] == M.demand[s,m])


model1.total_demand = Constraint(model1.s, model1.m, rule=total_demand)

# Outsourcing Regulation

model1.outsource_demand = ConstraintList()
for m in model1.m:
    for s in model1.s:
        if s == 'A':
            model1.outsource_demand.add(expr = model1.Outsource[s,m] <= 0.3*model1.demand[s,m])
        elif s == 'B':
            model1.outsource_demand.add(expr = model1.Outsource[s,m] <= 0.4*model1.demand[s,m])

In [86]:
# Objective function

model1.value = Objective(expr = sum(sum(model1.FTE[s,m]*FTE_Monthly_Salary[s,m]
                                        + model1.Outsource[s,m]*UnitOutSourceCost[s,m] for s in model1.s) for m in model1.m),
                         sense= minimize)

In [87]:
# Invoking the solver

result = SolverFactory('glpk').solve(model1)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 12196780.4410867
  Upper bound: 12196780.4410867
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0585939884185791
# ----------------------------------------------------------
#   Solution Information
# -------------

In [88]:
# Creating dataframe for the results

model1.pprint()

8 Set Declarations
    FTE_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    s*m :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Outsource_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    s*m :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B

In [89]:
# Print the value of the objective function

optimized_cost_worst = round(model1.value()/1000000,2)
optimized_cost_worst

12.2

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [91]:
#Creating an empty list

Output_Worst = []

# For every iteration, the code below will create a list containing 9 values - State, Month, Demand,No of FTE,
#No of Outsource,No of FTE Processed Applications,Percent FTE Applications,Percent Outsourced Applications,Estimated Cost,
#Average Cost per Application

for s in model1.s:
    for m in model1.m:
        
        
        Demand = model1.demand[s, m]
        No_of_FTE=model1.FTE[s,m].value
        No_of_Outsource = model1.Outsource[s,m].value
        No_of_FTE_Processed_App = model1.FTE[s,m].value*40*model1.sa[s,m]
        Percent_FTE_App = round((No_of_FTE/Demand)*100,2)
        Percent_Outsourced_App = round((No_of_Outsource/Demand)*100,2)
        Estimated_Cost = model1.FTE[s,m].value*FTE_Monthly_Salary[s,m] + model1.Outsource[s,m].value*UnitOutSourceCost[s,m]
        Cost_Per_App = round(Estimated_Cost/Demand,2)
        
        Output_Worst.append([s,m,Demand,No_of_FTE,No_of_Outsource,No_of_FTE_Processed_App,Percent_FTE_App, 
                              Percent_Outsourced_App,Estimated_Cost,Cost_Per_App])
        
print(Output_Worst)

[['A', 'Jan', 1998, 71.3571428571429, 0.0, 1998.0000000000011, 3.57, 0.0, 356785.7142857145, 178.57], ['A', 'Feb', 1998, 53.8076923076923, 599.0, 1398.9999999999998, 2.69, 29.98, 376858.4615384615, 188.62], ['A', 'Mar', 1998, 71.3571428571429, 0.0, 1998.0000000000011, 3.57, 0.0, 356785.7142857145, 178.57], ['A', 'Apr', 1998, 66.6, 0.0, 1998.0, 3.33, 0.0, 333000.0, 166.67], ['A', 'May', 1998, 71.3571428571429, 0.0, 1998.0000000000011, 3.57, 0.0, 356785.7142857145, 178.57], ['A', 'Jun', 1998, 53.8076923076923, 599.0, 1398.9999999999998, 2.69, 29.98, 376858.4615384615, 188.62], ['A', 'Jul', 1998, 58.2916666666667, 599.0, 1399.0000000000007, 2.92, 29.98, 399278.3333333335, 199.84], ['A', 'Aug', 1998, 53.8076923076923, 599.0, 1398.9999999999998, 2.69, 29.98, 376858.4615384615, 188.62], ['A', 'Sep', 1998, 71.3571428571429, 0.0, 1998.0000000000011, 3.57, 0.0, 356785.7142857145, 178.57], ['A', 'Oct', 1998, 53.8076923076923, 599.0, 1398.9999999999998, 2.69, 29.98, 376858.4615384615, 188.62], ['

In [92]:
# Creating dataframe for the results

Output_Worst = pd.DataFrame(Output_Worst, columns = ['State','Month','Demand','No of FTE','No of Outsource',
                                                       'No of FTE Processed Applications','Percent FTE Applications',
                                                       'Percent Outsourced Applications','Estimated Cost',
                                                       'Cost per Application'])

Output_Worst[['Estimated Cost in mn $']] = Output_Worst[['Estimated Cost']]/1000000

Output_Worst['Demand'] = Output_Worst['Demand'].astype(int)

Output_Worst['No of Outsource'] = Output_Worst['No of Outsource'].astype(int)

Output_Worst['No of FTE Processed Applications'] = Output_Worst['No of FTE Processed Applications'].astype(int)

Output_Worst = Output_Worst.round({"No of FTE":1, "No of Outsource":0, "Estimated Cost in mn $":1})

In [93]:
#Printing top 5 Rows of Final dataframe

Output_Worst.head()

,State,Month,Demand,No of FTE,No of Outsource,No of FTE Processed Applications,Percent FTE Applications,Percent Outsourced Applications,Estimated Cost,Average Cost per Application,Estimated Cost in mn $
0,A,Jan,1998,71.4,0,1998,3.57,0.00,356785.714286,178.57,0.4
1,A,Feb,1998,53.8,599,1398,2.69,29.98,376858.461538,188.62,0.4
2,A,Mar,1998,71.4,0,1998,3.57,0.00,356785.714286,178.57,0.4
3,A,Apr,1998,66.6,0,1998,3.33,0.00,333000.000000,166.67,0.3
4,A,May,1998,71.4,0,1998,3.57,0.00,356785.714286,178.57,0.4


#### 3.1.1 Optimal number of staff members 

In [ ]:
# write your code here

Optimal_Staff_Members = round(Output_Worst['No of FTE'].mean(),1)

print("Optimal Staff Members : ", Optimal_Staff_Members)

#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here

Total_Outsourcing =  sum(Output_Worst['No of Outsource'])

Total_Demand =  sum(Output_Worst['Demand'])

Percent_Outsourced_App_Total = round((Total_Outsourcing/Total_Demand)*100,1)

print("Total Outsourced Application % is : ", Percent_Outsourced_App_Total)

#### 3.1.3 Average cost per application

In [ ]:
# write your code here

Total_Cost = sum(Output_Worst['Estimated Cost'])

Total_Demand =  sum(Output_Worst['Demand'])

Average_Cost_Per_App = round(Total_Cost/Total_Demand,2)

print("Average Cost per Application: ", Average_Cost_Per_App)

In [ ]:
# Writing the results in to an Excel sheet

with pd.ExcelWriter('Output_Worst_Q3.xlsx') as writer:
    Output_Worst.to_excel(writer)

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance

model2 = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters

model2.i = Set(initialize=State.tolist(),doc='States')

model2.j = Set(initialize=Month.tolist(),doc='Months')

model2.demand = Param(model2.i,model2.j,initialize = Demand,doc='Demand')

model2.sa = Param(model2.i,model2.j,initialize = StaffAv_UB,doc = 'StaffUpperBoundPercent')

In [ ]:
# Decision variables

model2.x = Var(model2.i,model2.j,doc='No of FTE',domain = NonNegativeReals)

model2.y = Var(model2.i,model2.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [ ]:
# Constraints

# Demand Constraint

def total_demand(m, i, j):
    return (m.x[i, j] * 40 * m.sa[i, j] + m.y[i, j] == m.demand[i, j])


model2.total_demand = Constraint(model2.i, model2.j, rule=total_demand)

# Outsourcing Regulation

model2.outsource_demand = ConstraintList()
for j in model2.j:
    for i in model2.i:
        if i == 'A':
            model2.outsource_demand.add(expr = model2.y[i,j] <= 0.3*model2.demand[i,j])
        elif i == 'B':
            model2.outsource_demand.add(expr = model2.y[i,j] <= 0.4*model2.demand[i,j])

In [ ]:
# Objective function

model2.value = Objective(expr = sum(sum(model2.x[i,j]*FTE_Salary[i,j]+ 
                                        model2.y[i,j]*UnitOutSourceCost[i,j] for i in model2.i) for j in model2.j),
                         sense= minimize)


In [ ]:
# Invoking the solver

result = SolverFactory('glpk').solve(model2)
result.write()

In [ ]:
#Printing the Model

model2.pprint()

In [ ]:
# Print the value of the objective function

optimized_cost_best = round(model2.value()/1000000,2)
optimized_cost_best

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
#Creating an empty list

Output_best = []

# For every iteration, the code below will create a list containing 9 values - State, Month, No of FTE,Demand,
#No of outsource app,No of FTE processed App,Estimated Cost,percent outsourced app,average cost per application

for i in model2.i:
    for j in model2.j:
        # cost for the application approval process
        no_of_fte = model2.x[i, j].value
        demand = model2.demand[i, j]
        no_of_outsource_app = model2.y[i, j].value
        no_of_FTE_processed_app = model2.x[i,j].value*40*model2.sa[i,j]
        cost = model2.x[i, j].value*FTE_Salary[i, j] + model2.y[i, j].value*UnitOutSourceCost[i, j]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        avg_cost_per_app = round(cost/demand,1)

        Output_best.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, 
                            cost, percent_outsourced_app,avg_cost_per_app])

        print(Output_best)

In [ ]:
# Creating dataframe for the results

Output_best = pd.DataFrame(Output_best, columns = ['State', 'Month', 'No of FTE Best',"Demand",
                                                   'No of Outsource App Best',"No of FTE Processed App Best", 
                                                   'Estimated Cost Best', 'Percent Outsourced App Best',  
                                                   'Average Cost Per Application Best'])


Output_best[['Estimated Cost in mn $ Best']] = Output_best[['Estimated Cost Best']]/1000000

Output_best['Demand'] = Output_best['Demand'].astype(int)

Output_best['No of Outsource App Best'] = Output_best['No of Outsource App Best'].astype(int)

Output_best['No of FTE Processed App Best'] = Output_best['No of FTE Processed App Best'].astype(int)

Output_best = Output_best.round({"No of FTE Best":1, "No of Outsource App Best":0, "Estimated Cost in mn $ Best":1})

In [ ]:
#Printing top 5 Rows of Final dataframe

Output_best.head()

In [ ]:
#Various Parameters

total_outsourcing =  sum(Output_best['No of Outsource App Best'])

total_cost = sum(Output_best['Estimated Cost Best'])

optimal_staff_members = round(Output_best['No of FTE Best'].mean(),1)

total_demand =  sum(Output_best['Demand'])

average_cost_per_appln = round(total_cost/total_demand,2)

percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)

print("Total Outsourced Application: ", total_outsourcing)

print("Total Demand Application: ", total_demand)

print("Optimal Staff Members : ", optimal_staff_members)

#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here

print("Total Outsourced Application % is : ", percent_outsourced_app_total)

#### 3.2.3 Average cost per application

In [ ]:
# write your code here

print("Average Cost per Application: ", average_cost_per_appln)

In [ ]:
# Writing the results in to an Excel sheet

with pd.ExcelWriter('Output_Best_Q3.xlsx') as writer:
    Output_best.to_excel(writer)

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [ ]:
df = Output_actual[['Month', 'Demand', 'No of Outsource App','No of FTE processed App','Percent Outsourced App']].groupby(['Month']).sum()
df['Percentage of FTE Processed App'] = df['No of FTE processed App']/df['Demand']*100
ax = df[['Percent Outsourced App','Percentage of FTE Processed App']].plot(kind='bar', stacked=True, figsize=(10,5))
ax.set_ylabel("Percentage", fontsize=12)
plt.title("Stacked bar chart between percentages of FTE Processed app and Outsourced app", fontsize=12)
plt.show()

In [ ]:
#Creating the chart to show cost per application in every case
df_new = pd.concat([Output_actual[['Average Cost Per Application','Month']], Output_worst['Average Cost Per Application Worst'], Output_best['Average Cost Per Application Best']], axis=1)
df_new = df_new.groupby(['Month']).mean()
df_new.plot.line(figsize=[10,5])
plt.xlabel("Month")
plt.ylabel("Cost")
plt.show()